In [19]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session
import glob
import json
import os
import urllib2
from urllib2 import Request, urlopen

In [20]:
requests_cache.install_cache("cache")

def geonames_search(geonameid):
    base = 'http://api.geonames.org/childrenJSON'
    username = 'stephanielamm'
    
    params = ({"geonameId": geonameid, "username":username})
    
    response = base + "?" + "&".join("{}={}".format(a,b) for a,b in params.items())

    collreq = Request(response)
    collf = urllib2.urlopen(collreq)
    college_json = json.loads(collf.read())

    urllist = [i for i in college_json.values()]
        
    return urllist

In [43]:
def get_city_name(us_state):
    us_id = 6252001
    country = geonames_search('%d' %us_id)
    
    for i in range(0, len(country[1])):
        if country[1][i]['name'] == us_state:
            geonameid = country[1][i]['geonameId']
            states = geonames_search('%d' %geonameid)
    
    id_list = []
    #from state, extract the id of all county
    for i in range(0, len(states[1])):
        idnum = states[1][i]['geonameId']
        id_list.append(idnum)

    #extract all county info in CA
    city_list = []
    for i in range(0, len(id_list)):
        region = geonames_search('%d' %id_list[i])
        city_list.append(region)
        
    #from county, extract cities/town with more tan 10000 population, lat, and lng 
    citynames = []
    lat_list = []
    lng_list = []
    for i in range(0, len(city_list)):
        for j in range(0,len(city_list[i][1])):
            if city_list[i][1][j]['fcl'] == 'P' and city_list[i][1][j]['population'] > 10000:
                names = city_list[i][1][j]['name']
                lat = city_list[i][1][j]['lat']
                lng = city_list[i][1][j]['lng']
                citynames.append(names)
                lat_list.append(lat)
                lng_list.append(lng)

    citynames = pd.DataFrame(citynames)
    lat_list = pd.DataFrame(lat_list)
    lng_list = pd.DataFrame(lng_list)
    df = pd.concat([citynames, lat_list, lng_list], axis = 1)
    df.columns = ['City', 'Latitude', 'Longitude']

    return df

In [44]:
city_df = get_city_name("California")

In [45]:
city_df

,City,Latitude,Longitude
0,Alameda,37.76521,-122.24164
1,Albany,37.88687,-122.29775
2,Ashland,37.69465,-122.11385
3,Berkeley,37.87159,-122.27275
4,Castro Valley,37.6941,-122.08635
5,Cherryland,37.67938,-122.1033
6,Dublin,37.70215,-121.93579
7,Emeryville,37.83132,-122.28525
8,Fairview,37.67854,-122.0458
9,Fremont,37.54827,-121.98857


In [46]:
requests_cache.install_cache("cache")

def yelp_search(term, location):
    """
    makes a search request
    
    Argument: search term, location
    
    Return: search result items as a list (for 0 items, return an empty list)
    """
    consumer_key = "uFKYftN0fu4pD6guW2cYng"
    consumer_secret = "iUYQ2Hk3-vTQVd0zOFnvloWPivA"
    token = "FZnep4FiFmwrrifuNm6M-_K661Wb3J02"
    token_secret = "CRvBo_fO-r4pE_-p2UrnN_mxBlc"
   
    urlbase = "http://api.yelp.com/v2/search"
    params = ({"term": term})
    params.update({"location": location, "sort": "1", "cll" : "latitude,longitude"})
    session = OAuth1Session(consumer_key, consumer_secret,token,token_secret)
     
    request = session.get(urlbase,params=params)
   
    #Transforms JSON API response into dictionary
    data = request.json()
    session.close()
    #data_dict = data["businesses"]
    
    return data

In [ ]:
data_list = []
for i in range(0, len(city_df['City'])):
    data = yelp_search("food", "%s" %city_df['City'][i])
    data_list.append(data)

In [ ]:
data_list

In [ ]:
data = yelp_search("food", "Sacramento")
data

In [ ]:
pd.DataFrame(data['businesses'])

In [ ]:
path = "/Users/Stephanie/Desktop/yelp_dataset_challenge_round9"
files = glob.glob(os.path.join(path,"*.json"))
files

In [ ]:
############# Copy from online(not sure if it works or not yet) ##########

#Convert Yelp Academic Dataset from JSON to CSV
import csv
def convert(line):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    data = json.loads(line)
    for k, v in data.items():
        if isinstance(v, list):
            data[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                data['%s_%s' % (k, kk)] = vv
            del data[k]
    return line


csv_filename = 'review.csv'
df = pd.DataFrame([convert(line) for line in file(files[2])])
df.to_csv(csv_filename, index=False)


In [ ]:
def readfile(files):
    # read the file into a python array
    with open(files, 'rb') as f:
        data = f.readlines()
        # remove the trailing "\n" from each line
        data = map(lambda x: x.rstrip(), data)
        data_str = "[" + ','.join(data) + "]"
        df = pd.read_json(data_str)
        
        f.close()
        return df
     
def fixdf(newlist):
    new_df = pd.concat(newlist, axis = 0)
    return new_df

yelplist = []

In [ ]:
review_data = readfile(files[2])

In [ ]:
review_data.head()